In [3]:

!pip3 install datasets
!pip3 install sentencepiece
!pip3 install seqeval
!pip install transformers[torch]
!pip install accelerate
!pip install transformers
from datasets import load_dataset
import torch
from sklearn.metrics import f1_score
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments,DataCollatorForTokenClassification
import accelerate
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score



2024-03-11 18:27:48.338847: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 18:27:48.338962: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 18:27:48.446623: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Naamapadam corpus

In [4]:
from datasets import load_dataset
data = load_dataset('ai4bharat/naamapadam', 'hi')


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset naamapadam_pr downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___naamapadam_pr/hi/1.0.0/99b5ec77eabfaa3fbff510d8cf70d7c34519486cb7dbee99ede19474ddff9b20. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
trainData = data['train']
valData = data['validation']
testData = data['test']

In [6]:

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")

model = AutoModelForTokenClassification.from_pretrained("ai4bharat/IndicNER")

tokenizer_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/667M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [7]:
data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 985787
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 867
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 13460
    })
})

In [8]:
def prepare_features(examples):
    tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Special token for "ignore"
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx])  # Carry over label for subwords
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [9]:

trainData = trainData.map(prepare_features, batched=True)
testData = testData.map(prepare_features, batched=True)
valData = valData.map(prepare_features, batched=True)

  0%|          | 0/986 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

In [10]:
testData

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 867
})

Fine-Tune with the Trainer API

In [11]:
# def compute_metrics(p):
#     predictions, labels = p.predictions, p.label_ids
#     predictions = np.argmax(predictions, axis=2).flatten()  # Convert probabilities to class labels
#     labels = labels.flatten()
#     # Calculate F1 score (macro averaging for NER)
#     f1 = f1_score(labels, predictions, average="macro")
#     return {"macro_f1": f1}
def compute_metrics(p):
    predictions, labels = p.predictions, p.label_ids

    # Flatten the predictions and labels
    predictions_flat = np.argmax(predictions, axis=2).flatten()
    labels_flat = labels.flatten()

    # Remove occurrences of -100 from both predictions and labels
    mask = (labels_flat != -100)
    predictions_filtered = predictions_flat[mask]
    labels_filtered = labels_flat[mask]

    # Calculate macro F1 score
    f1 = f1_score(labels_filtered, predictions_filtered, average="macro")

    # Calculate precision, recall, and accuracy with zero_division parameter
    precision = precision_score(labels_filtered, predictions_filtered, average="macro", zero_division=1)
    recall = recall_score(labels_filtered, predictions_filtered, average="macro")
    accuracy = accuracy_score(labels_filtered, predictions_filtered)

    return {
        "macro_f1": f1,
        "precision": precision,
        "recall": recall,
        "accuracy": accuracy
    }

In [12]:
model=model.to("cuda")
print(model)


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

In [13]:
train_1 = trainData.shuffle(seed=42).select(range(30000))

In [14]:

data_collator = DataCollatorForTokenClassification(tokenizer)
training_args = TrainingArguments(
    output_dir="./indic_ner_fine_tuned",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir="./indic_ner_logs",
    evaluation_strategy="steps",  # Evaluate every epoch
    eval_steps=500,  # Adjust based on your training steps
    metric_for_best_model="macro_f1",  # Specify the metric for early stopping
    save_total_limit = 2

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_1,
    eval_dataset=valData,
    compute_metrics=compute_metrics,  # Use the custom metrics function
    data_collator=data_collator
)

accelerator = accelerate.Accelerator()


trainNer = trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,Macro F1,Precision,Recall,Accuracy
500,0.406500,0.241796,0.828687,0.849053,0.810782,0.925989
1000,0.224400,0.226584,0.834325,0.835101,0.834304,0.927640
1500,0.220200,0.232453,0.833206,0.834716,0.833332,0.927621
2000,0.203100,0.240768,0.835622,0.836832,0.835491,0.927978
2500,0.156600,0.236515,0.835081,0.838254,0.832803,0.927876
3000,0.152300,0.235686,0.834562,0.839855,0.830177,0.928058
3500,0.158900,0.236911,0.835954,0.842905,0.830523,0.928282
4000,0.134200,0.269422,0.833161,0.841959,0.825169,0.927172
4500,0.112900,0.261736,0.831564,0.836732,0.827208,0.926547
5000,0.110300,0.264448,0.836010,0.838965,0.833401,0.927961


In [15]:
trainer.evaluate(testData)

{'eval_loss': 0.2256152629852295,
 'eval_macro_f1': 0.8433462008091087,
 'eval_precision': 0.8459805228289533,
 'eval_recall': 0.8469770832630077,
 'eval_accuracy': 0.9347412151635797,
 'eval_runtime': 2.9443,
 'eval_samples_per_second': 294.466,
 'eval_steps_per_second': 18.68,
 'epoch': 3.0}

In [16]:

# from google.colab import drive
# drive.mount('/content/gdrive')

# save_path = '/content/gdrive/My Drive/MyModels/IndicNER'

# trainer.save_model(save_path)


In [17]:
# def get_predictions(model, dataset):
#     raw_predictions = trainer.predict(dataset)
#     predictions = np.argmax(raw_predictions.predictions, axis=2)
#     labels = raw_predictions.label_ids
#     return predictions, labels


In [18]:
# predictions, labels = get_predictions(model,testData)

# # Calculate the macro F1 score as in your training loop
# predictions = predictions.flatten()  # Convert probabilities to class labels
# labels = labels.flatten()

# f1 = f1_score(labels, predictions, average="macro")
# print(f"Macro F1 on Test Set: {f1}")

# # Additional metrics (optional)
# from sklearn.metrics import classification_report
# report = classification_report(labels, predictions)
# print(report)
# print(labels)

In [19]:
# train_2 = trainData.shuffle(seed=42).select(range(30000))

In [24]:
from transformers import AutoConfig
config_bert = AutoConfig.from_pretrained('ai4bharat/indic-bert', num_labels=7, finetuning_task='ner')
tokenizer_bert = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model_bert = AutoModelForTokenClassification.from_pretrained('ai4bharat/indic-bert', num_labels=7 )
# print(model_bert)
model_bert=model_bert.to("cuda")
# print(model_bert)

Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AlbertForTokenClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(200000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, b

In [25]:
data_collator_bert = DataCollatorForTokenClassification(tokenizer_bert)
training_args_bert = TrainingArguments(
    output_dir="./indic_bert_fine_tuned",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir="./indic_bert_logs",
    evaluation_strategy="steps",  # Evaluate every epoch
    eval_steps=500,  # Adjust based on your training steps
    metric_for_best_model="macro_f1",  # Specify the metric for early stopping
    save_total_limit = 2

)

trainer_bert = Trainer(
    model=model_bert,
    args=training_args_bert,
    train_dataset=train_1,
    eval_dataset=valData,
    compute_metrics=compute_metrics,  # Use the custom metrics function
    data_collator=data_collator_bert
)

accelerator = accelerate.Accelerator()


train_bert = trainer_bert.train()

Step,Training Loss,Validation Loss,Macro F1,Precision,Recall,Accuracy
500,0.882600,0.755238,0.294485,0.723430,0.250022,0.768237
1000,0.660300,0.615878,0.459078,0.688088,0.420878,0.808318
1500,0.553900,0.513221,0.581776,0.704554,0.531885,0.839059
2000,0.491100,0.478587,0.619992,0.680979,0.597122,0.847821
2500,0.434400,0.444154,0.651283,0.709638,0.613842,0.858706
3000,0.417500,0.434524,0.657169,0.710247,0.626356,0.860676
3500,0.413700,0.415442,0.679788,0.716113,0.656855,0.867042
4000,0.371100,0.409678,0.690059,0.699959,0.682018,0.868382
4500,0.335300,0.402828,0.694747,0.709027,0.682794,0.870546
5000,0.325300,0.399252,0.694591,0.749947,0.655638,0.875379


Checkpoint destination directory ./indic_bert_fine_tuned/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.


In [26]:
trainer_bert.evaluate(testData)

{'eval_loss': 0.3567679524421692,
 'eval_macro_f1': 0.7133476792867718,
 'eval_precision': 0.7503896223567134,
 'eval_recall': 0.6848482136519672,
 'eval_accuracy': 0.8858692516300272,
 'eval_runtime': 2.9862,
 'eval_samples_per_second': 290.333,
 'eval_steps_per_second': 18.418,
 'epoch': 3.0}

In [27]:
trainer.save_model('./IndicNER')
trainer_bert.save_model('./IndicBERT')